In [2]:
#This session is for reading data
import gzip
from collections import defaultdict 
import json

data_modcloth = []
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l

data_rent_the_way = []
for l in readJSON("renttherunway_final_data.json.gz"):
    data_rent_the_way.append(l)
# print(data_rent_the_way[0])


one_rent_the_way = json.loads(data_rent_the_way[0])
print(list(one_rent_the_way.keys()))

['fit', 'user_id', 'bust size', 'item_id', 'weight', 'rating', 'rented for', 'review_text', 'body type', 'review_summary', 'category', 'height', 'size', 'age', 'review_date']


do one hot encoding - body type, rented for, category

In [22]:
#This session is one hot encoding code
import numpy as np

body_t_count = defaultdict(int)
rent_for_count = defaultdict(int)
category_count = defaultdict(int)

count_missing_bt = 0 
count_missing_rent_for = 0

for d in data_rent_the_way:
    one_interaction = json.loads(d)

    if('body type' in one_interaction):
        body_t_count[one_interaction['body type']] += 1
    else:
        count_missing_bt += 1
    if('rented for' in one_interaction):
        rent_for_count[one_interaction['rented for']] += 1
    else:
        count_missing_rent_for += 1
    category_count[one_interaction['category']] += 1


body_t_one_hot = defaultdict(int)
rf_one_hot = defaultdict(int)
category_one_hot = defaultdict(int)



print(count_missing_bt, " of body type count is missing")
print(count_missing_rent_for, " of rent_for are missing")


# one hot encoding for boudy type
bt_keys = list(body_t_count.keys())
bt_one_hot_to_orig = defaultdict(str)

total_bt = len(body_t_count)
for i in range(len(bt_keys)):
    one_hot = np.zeros(len(body_t_count) - 1, dtype=int)
    if(i != total_bt - 1):
        one_hot[i] = 1
    
    str_one_hot = ''.join(map(str, one_hot))
    body_t_one_hot[bt_keys[i]] =  str_one_hot
    bt_one_hot_to_orig[str_one_hot] = bt_keys[i]

print(body_t_one_hot)


#one hot encoding for rent for
rf_keys = list(rent_for_count.keys())
rf_one_hot_to_orig = defaultdict(str)

total_rf = len(rent_for_count)
for i in range(len(rf_keys)):
    one_hot = np.zeros(len(rent_for_count) - 1, dtype=int)
    if(i != total_rf - 1):
        one_hot[i] = 1
    
    str_one_hot = ''.join(map(str, one_hot))
    rf_one_hot[rf_keys[i]] =  str_one_hot
    rf_one_hot_to_orig[str_one_hot] = rf_keys[i]

#one hot encoding for catagory 
cat_keys = list(category_count.keys())
cat_one_hot_to_orig = defaultdict(str)

total_cat = len(category_count)
for i in range(len(cat_keys)):
    one_hot = np.zeros(len(category_count) - 1, dtype=int)
    if(i != total_cat - 1):
        one_hot[i] = 1
    
    str_one_hot = ''.join(map(str, one_hot))
    category_one_hot[cat_keys[i]] =  str_one_hot
    cat_one_hot_to_orig[str_one_hot] = cat_keys[i]



14637  of body type count is missing
10  of rent_for are missing
defaultdict(<class 'int'>, {'straight & narrow': '100000', 'pear': '010000', 'athletic': '001000', 'hourglass': '000100', 'full bust': '000010', 'petite': '000001', 'apple': '000000'})


In [5]:
print(rf_one_hot['party'])
print(rf_one_hot_to_orig['01000000'])

01000000
party


In [23]:
#Function you can call for One hot encoding


# @return the one hot encoding of the body type
# @param: type: string     the body type that are in the renttherunaway database
def get_body_type_one_hot(bodyTypeName):
    return body_t_one_hot[bodyTypeName]

# @return: the body type that are in the renttherunaway database
# @param type: string    the one hot encoding of the body type
def body_type_one_hot_to_original(one_hot: str):
    return bt_one_hot_to_orig[one_hot]

# @return the one hot encoding of the rent for 
# @param: type: string     the rent for that are in the renttherunaway database
def rent_for_one_hot(rentForType):
    return rf_one_hot[rentForType]

# @return: the rent for that are in the renttherunaway database
# @param type: string    the one hot encoding of the rent for
def rent_for_one_hot_to_original(one_hot: str):
    return rf_one_hot_to_orig[one_hot]


# @return the one hot encoding of the catagory
# @param: type: string     the catagory for that are in the renttherunaway database
def catagory_for_one_hot(catType):
    return category_one_hot[catType]

# @return: the rent for that are in the renttherunaway database
# @param type: string    the one hot encoding of the rent for
def catagory_one_hot_to_original(one_hot: str):
    return cat_one_hot_to_orig[one_hot]

In [26]:
print(cat_keys)
print(catagory_for_one_hot('culottes'))
print(catagory_one_hot_to_original("0000000000000000000000000000000010000000000000000000000000000000000"))

['gown', 'sheath', 'dress', 'leggings', 'top', 'jumpsuit', 'sweater', 'jacket', 'shirtdress', 'maxi', 'shift', 'romper', 'pants', 'shirt', 'mini', 'skirt', 'pullover', 'blouse', 'suit', 'coat', 'trench', 'bomber', 'cape', 'blazer', 'vest', 'duster', 'ballgown', 'tank', 'poncho', 'frock', 'tunic', 'cardigan', 'culottes', 'down', 'trouser', 'midi', 'pant', 'legging', 'print', 'knit', 'culotte', 'sweatshirt', 'peacoat', 'kaftan', 'overalls', 'jogger', 'tee', 'combo', 'henley', 'cami', 'blouson', 'turtleneck', 'trousers', 'overcoat', 'hoodie', 't-shirt', 'caftan', 'tight', 'kimono', 'for', 'crewneck', 'skirts', 'parka', 'buttondown', 'skort', 'sweatershirt', 'sweatpants', 'jeans']
0000000000000000000000000000000010000000000000000000000000000000000
culottes
